# Predicting COVID-19 in European Countries with Random Forest

In [19]:
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
from datetime import datetime
from pandas.plotting import lag_plot
from matplotlib import pyplot as plt
from matplotlib.dates import date2num
%matplotlib inline
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from category_encoders import TargetEncoder
import json
import math
plt.close("all")

euro_data = pd.read_csv("data/euro_countries_filled.csv", index_col=0)
with open("data/iso_country_codes.json", "r") as read_file:
    country_codes = json.load(read_file)

euro_data

,iso_code,location,date,new_cases,new_cases_smoothed,new_cases_per_million,new_cases_smoothed_per_million,population,new_tests,new_tests_smoothed,stringency_index,latitude,longitude
0,ALB,Albania,2019-12-31,0.0,0.000,0.000,0.000,2877800.0,0.0,0.0,0.00,41.0,20.0
1,ALB,Albania,2020-01-01,0.0,0.000,0.000,0.000,2877800.0,0.0,0.0,0.00,41.0,20.0
2,ALB,Albania,2020-01-02,0.0,0.000,0.000,0.000,2877800.0,0.0,0.0,0.00,41.0,20.0
3,ALB,Albania,2020-01-03,0.0,0.000,0.000,0.000,2877800.0,0.0,0.0,0.00,41.0,20.0
4,ALB,Albania,2020-01-04,0.0,0.000,0.000,0.000,2877800.0,0.0,0.0,0.00,41.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10487,UKR,Ukraine,2020-11-18,11968.0,11208.571,273.656,256.291,43733759.0,42862.0,41017.0,61.57,49.0,32.0
10488,UKR,Ukraine,2020-11-19,12496.0,11477.857,285.729,262.448,43733759.0,40585.0,40110.0,61.57,49.0,32.0
10489,UKR,Ukraine,2020-11-20,13357.0,11806.429,305.416,269.961,43733759.0,NaN,NaN,61.57,49.0,32.0
10490,UKR,Ukraine,2020-11-21,29155.0,14287.571,666.647,326.694,43733759.0,NaN,NaN,61.57,49.0,32.0


## Preprocessing

In [2]:
# Remove rows before 2020-03-01 and after 2020-11-07
start_date, end_date = '2020-03-01', '2020-11-07'

df = euro_data
df['date'] = pd.to_datetime(df['date'])
df = df[~(df['date'] < start_date)]
df = df[~(df['date'] > end_date)]

dates = df['date']

# Drop unneeded columns
df = df.drop(['location', 'latitude', 'longitude', 'new_tests', 'new_tests_smoothed', 'population'], axis = 1)

dfs = [group[1] for group in df.groupby('iso_code')]

dfs[0]

,iso_code,date,new_cases,new_cases_smoothed,new_cases_per_million,new_cases_smoothed_per_million,stringency_index
61,ALB,2020-03-01,0.0,0.000,0.000,0.000,0.00
62,ALB,2020-03-02,0.0,0.000,0.000,0.000,0.00
63,ALB,2020-03-03,0.0,0.000,0.000,0.000,0.00
64,ALB,2020-03-04,0.0,0.000,0.000,0.000,0.00
65,ALB,2020-03-05,0.0,0.000,0.000,0.000,0.00
...,...,...,...,...,...,...,...
308,ALB,2020-11-03,321.0,296.857,111.544,103.154,50.93
309,ALB,2020-11-04,381.0,310.714,132.393,107.969,50.93
310,ALB,2020-11-05,396.0,322.857,137.605,112.189,50.93
311,ALB,2020-11-06,421.0,343.714,146.292,119.436,50.93


In [3]:
for i in range(len(dfs)):
    dfs[i]['day'] = df['date'].dt.day
    dfs[i]['month'] = df['date'].dt.month
    dfs[i]['year'] = df['date'].dt.year
    
dfs[0]

,iso_code,date,new_cases,new_cases_smoothed,new_cases_per_million,new_cases_smoothed_per_million,stringency_index,day,month,year
61,ALB,2020-03-01,0.0,0.000,0.000,0.000,0.00,1,3,2020
62,ALB,2020-03-02,0.0,0.000,0.000,0.000,0.00,2,3,2020
63,ALB,2020-03-03,0.0,0.000,0.000,0.000,0.00,3,3,2020
64,ALB,2020-03-04,0.0,0.000,0.000,0.000,0.00,4,3,2020
65,ALB,2020-03-05,0.0,0.000,0.000,0.000,0.00,5,3,2020
...,...,...,...,...,...,...,...,...,...,...
308,ALB,2020-11-03,321.0,296.857,111.544,103.154,50.93,3,11,2020
309,ALB,2020-11-04,381.0,310.714,132.393,107.969,50.93,4,11,2020
310,ALB,2020-11-05,396.0,322.857,137.605,112.189,50.93,5,11,2020
311,ALB,2020-11-06,421.0,343.714,146.292,119.436,50.93,6,11,2020


## Detrend the Time Series

### Differencing

In [4]:
# Difference all countries
for i in range(len(dfs)):
    dfs[i]['new_cases_diff'] = dfs[i]['new_cases_smoothed_per_million'].diff(periods=7)
dfs[0]

,iso_code,date,new_cases,new_cases_smoothed,new_cases_per_million,new_cases_smoothed_per_million,stringency_index,day,month,year,new_cases_diff
61,ALB,2020-03-01,0.0,0.000,0.000,0.000,0.00,1,3,2020,NaN
62,ALB,2020-03-02,0.0,0.000,0.000,0.000,0.00,2,3,2020,NaN
63,ALB,2020-03-03,0.0,0.000,0.000,0.000,0.00,3,3,2020,NaN
64,ALB,2020-03-04,0.0,0.000,0.000,0.000,0.00,4,3,2020,NaN
65,ALB,2020-03-05,0.0,0.000,0.000,0.000,0.00,5,3,2020,NaN
...,...,...,...,...,...,...,...,...,...,...,...
308,ALB,2020-11-03,321.0,296.857,111.544,103.154,50.93,3,11,2020,-0.844
309,ALB,2020-11-04,381.0,310.714,132.393,107.969,50.93,4,11,2020,4.815
310,ALB,2020-11-05,396.0,322.857,137.605,112.189,50.93,5,11,2020,8.340
311,ALB,2020-11-06,421.0,343.714,146.292,119.436,50.93,6,11,2020,16.927


## Feature Engineering

In [5]:
for i in range(len(dfs)):
    # Remove date column
    dfs[i] = dfs[i].drop('date', axis = 1)

In [6]:
lag = 3

def build_lagged_features(s,lag=3,dropna=True):
    if type(s) is pd.DataFrame:
        new_dict={}
        for col_name in s:
            new_dict[col_name]=s[col_name]
            # create lagged Series
            for l in range(1,lag+1):
                new_dict['%s_lag%d' %(col_name,l*7)]=s[col_name].shift(l*7)
        res=pd.DataFrame(new_dict,index=s.index)
    if dropna:
        return res.dropna()
    else:
        return res

def build_all(features):
    new_features = []
    for f in features:
        f = f.dropna(subset=['new_cases_diff'])
        lagged = build_lagged_features(f.drop(['day', 'month', 'year', 'iso_code'], axis = 1), lag=lag)
        lagged['day'] = f['day'][lag:]
        lagged['month'] = f['month'][lag:]
        lagged['year'] = f['year'][lag:]
        #lagged = lagged.drop(['total_tests', 'new_tests_smoothed'], axis = 1)
        lagged['iso_code'] = f['iso_code'][lag:]
        new_features.append(lagged)
    return new_features

# Create lags
features = build_all(dfs)
features = [x for x in features if len(x) > 7]

features[0]

,new_cases,new_cases_lag7,new_cases_lag14,new_cases_lag21,new_cases_smoothed,new_cases_smoothed_lag7,new_cases_smoothed_lag14,new_cases_smoothed_lag21,new_cases_per_million,new_cases_per_million_lag7,...,stringency_index_lag14,stringency_index_lag21,new_cases_diff,new_cases_diff_lag7,new_cases_diff_lag14,new_cases_diff_lag21,day,month,year,iso_code
89,11.0,6.0,5.0,0.0,17.286,5.429,5.429,0.000,3.822,2.085,...,81.48,0.00,4.121,0.000,1.886,0.000,29,3,2020,ALB
90,15.0,13.0,4.0,2.0,17.571,6.714,5.714,0.000,5.212,4.517,...,81.48,36.11,3.773,0.347,1.986,0.000,30,3,2020,ALB
91,11.0,11.0,9.0,4.0,17.571,7.000,6.429,0.000,3.822,3.822,...,81.48,41.67,3.674,0.198,2.234,0.000,31,3,2020,ALB
92,20.0,23.0,4.0,4.0,17.143,9.714,6.429,0.000,6.950,7.992,...,81.48,51.85,2.581,1.142,2.234,0.000,1,4,2020,ALB
93,16.0,23.0,4.0,1.0,16.143,12.429,6.857,0.000,5.560,7.992,...,81.48,51.85,1.290,1.936,2.383,0.000,2,4,2020,ALB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,321.0,288.0,295.0,171.0,296.857,299.286,254.286,165.714,111.544,100.076,...,54.63,54.63,-0.844,15.637,30.777,7.000,3,11,2020,ALB
309,381.0,284.0,301.0,182.0,310.714,296.857,271.286,169.143,132.393,98.686,...,54.63,54.63,4.815,8.886,35.493,6.652,4,11,2020,ALB
310,396.0,311.0,297.0,203.0,322.857,298.857,284.714,175.000,137.605,108.069,...,54.63,54.63,8.340,4.914,38.125,7.148,5,11,2020,ALB
311,421.0,275.0,302.0,257.0,343.714,295.000,291.143,187.571,146.292,95.559,...,54.63,54.63,16.927,1.340,35.990,10.921,6,11,2020,ALB


In [7]:
# Splitting into labels and features, training and testing sets
labels = {}
labels_diff = []

train_features = []
train_labels = []
test_features = {}
test_labels = {}
for i in range(len(features)):
    iso_code = features[i]['iso_code'].iloc[0]
    labels[iso_code] = features[i]['new_cases_smoothed_per_million']
    labels_diff.append(features[i]['new_cases_diff'])
    features[i] = features[i].drop(['new_cases', 'new_cases_smoothed', 'new_cases_per_million', 'new_cases_smoothed_per_million', 'new_cases_diff', 'stringency_index'], axis = 1)
    # Using first 7 days of November as testing set
    train_features.append(features[i][:-7])
    train_labels.append(labels_diff[i][:-7])
    test_features[iso_code] = features[i][-7:]
    test_labels[iso_code] = labels[iso_code][-7:]

feature_list = list(features[0].columns)

In [8]:
# Target encode ISO code
iso_dict = {}
for i in range(len(train_features)):
    iso_code = train_features[i]['iso_code'].iloc[0]
    target_encoder = TargetEncoder(cols=['iso_code'], smoothing=8, min_samples_leaf=5).fit(train_features[i], train_labels[i])
    train_features[i] = target_encoder.transform(train_features[i])
    test_features[iso_code] = target_encoder.transform(test_features[iso_code])
    iso_dict[train_features[i]['iso_code'].iloc[0]] = iso_code

/home/alpakkan/.local/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/alpakkan/.local/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/alpakkan/.local/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/alpakkan/.local/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/alpakkan/.local/lib/python

## Training

In [9]:
# Convert to numpy array
np_train_features = np.array(pd.concat(train_features))
np_train_labels = np.array(pd.concat(train_labels))

In [10]:
# Train random forest model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 49)
rf.fit(np_train_features, np_train_labels)

RandomForestRegressor(n_estimators=1000, random_state=49)

In [11]:
diff_predictions = {}
for iso_code, tf in test_features.items():
    diff_predictions[iso_code] = rf.predict(np.array(tf))

In [12]:
from pmdarima.utils import diff_inv
predictions = {}
errors = {}
for iso_code, tl in test_labels.items():
    predictions[iso_code] = diff_inv(np.append(np.array(labels[iso_code].iloc[-14:-7]), diff_predictions[iso_code]), lag=7, differences=1)[-7:]
    # Get Mean Absolute Error (MAE)
    errors[iso_code] = abs(predictions[iso_code] - np.array(tl))
    print('MAE:', iso_code, round(np.mean(errors[iso_code]), 2), 'new cases per million')

MAE: ALB 16.67 new cases per million
MAE: AUT 43.95 new cases per million
MAE: BEL 468.74 new cases per million
MAE: BGR 51.16 new cases per million
MAE: BIH 92.92 new cases per million
MAE: BLR 9.67 new cases per million
MAE: CHE 103.26 new cases per million
MAE: CZE 215.21 new cases per million
MAE: DEU 21.22 new cases per million
MAE: DNK 13.02 new cases per million
MAE: ESP 57.93 new cases per million
MAE: EST 21.85 new cases per million
MAE: FIN 1.96 new cases per million
MAE: FRA 139.03 new cases per million
MAE: GRC 41.72 new cases per million
MAE: HRV 131.83 new cases per million
MAE: HUN 39.96 new cases per million
MAE: ITA 40.57 new cases per million
MAE: LTU 49.13 new cases per million
MAE: LVA 2.2 new cases per million
MAE: MDA 15.8 new cases per million
MAE: NLD 94.49 new cases per million
MAE: NOR 21.84 new cases per million
MAE: POL 44.96 new cases per million
MAE: PRT 28.28 new cases per million
MAE: ROU 31.47 new cases per million
MAE: RUS 9.57 new cases per million
MA

In [20]:
# Get MAE for entire Europe
predictions_eur = np.sum(np.array([*predictions.values()], dtype=object), 0)
labels_eur = np.sum(np.array([*test_labels.values()], dtype=object), 0)
errors_eur = abs(predictions_eur - labels_eur)

print('MAE:', round(np.mean(errors_eur), 2), 'new cases per million')
print('RMSE:', round(math.sqrt(np.mean(errors_eur**2)), 2))

MAE: 1387.46 new cases per million
RMSE: 1696.21


In [21]:
# Get average for entire Europe
round(np.mean([*errors.values()]), 2)
print('RMSE:', round(math.sqrt(np.mean([e**2 for e in errors.values()])), 2))

RMSE: 136.31


In [15]:
# Get feature importances
importances = list(rf.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort feature importances descendingly by importance
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: new_cases_diff_lag7  Importance: 0.72
Variable: iso_code             Importance: 0.07
Variable: month                Importance: 0.05
Variable: new_cases_smoothed_per_million_lag21 Importance: 0.02
Variable: new_cases_diff_lag14 Importance: 0.02
Variable: new_cases_diff_lag21 Importance: 0.02
Variable: day                  Importance: 0.02
Variable: new_cases_smoothed_lag21 Importance: 0.01
Variable: new_cases_per_million_lag7 Importance: 0.01
Variable: new_cases_smoothed_per_million_lag7 Importance: 0.01
Variable: new_cases_smoothed_per_million_lag14 Importance: 0.01
Variable: stringency_index_lag7 Importance: 0.01
Variable: stringency_index_lag21 Importance: 0.01
Variable: new_cases_lag7       Importance: 0.0
Variable: new_cases_lag14      Importance: 0.0
Variable: new_cases_lag21      Importance: 0.0
Variable: new_cases_smoothed_lag7 Importance: 0.0
Variable: new_cases_smoothed_lag14 Importance: 0.0
Variable: new_cases_per_million_lag14 Importance: 0.0
Variable: new_cases_

## Export results

In [22]:
d = {'rf_NOR': predictions['NOR'], 'rf_ESP': predictions['ESP'], 'rf_DEU': predictions['DEU'], 'rf_SWE': predictions['SWE']}
results = pd.DataFrame(d, index=dates.iloc[-7:])
results

,rf_NOR,rf_ESP,rf_DEU,rf_SWE
date,,,,
2020-11-01,33.149462,439.430615,137.011201,148.813723
2020-11-02,33.250056,481.545321,145.103321,155.084020
2020-11-03,38.137202,498.740626,169.566132,146.670207
2020-11-04,56.739377,504.257339,183.179320,156.448534
2020-11-05,63.672453,492.554453,193.526362,186.224650
2020-11-06,80.468194,528.041112,218.063007,240.153760
2020-11-07,90.960382,531.664918,232.341456,320.957430


In [23]:
results.to_csv('predictions/rf_predictions.csv')